In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.preprocessing.image as image
import matplotlib.pyplot as plt
import os

def dense_block(x, blocks, name, growth_rate = 32):
    for i in range(blocks):
        x = conv_block(x, growth_rate, name=name + '_block' + str(i + 1))
    return x

def transition_block(x, reduction, name):
    x = layers.BatchNormalization(axis=3, epsilon=1.001e-5,name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    filter = x.shape[3]
    x = layers.Conv2D(int(filter*reduction), 1,use_bias=False,name=name + '_conv')(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    x1 = layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(2 * growth_rate, 1,use_bias=False, name=name + '_1_conv')(x1)
    x1 = layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv2D(growth_rate, 3 ,padding='same',use_bias=False, name=name + '_2_conv')(x1)
    x = layers.Concatenate( name=name + '_concat')([x, x1])
    return x

def my_densenet():
    inputs = keras.Input(shape=(32, 32, 3), name='img')
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    blocks = [4,8,16]
    x = dense_block(x, blocks[0], name='conv1',growth_rate =32)
    x = transition_block(x, 0.5, name='pool1')
    x = dense_block(x, blocks[1], name='conv2',growth_rate =32)
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, blocks[2], name='conv3',growth_rate =32)
    x = transition_block(x, 0.5, name='pool3')
    x = layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='bn')(x)
    x = layers.Activation('relu', name='relu')(x)

    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(10, activation='softmax', name='fc1000')(x)

    model = keras.Model(inputs, x, name='densenet121')
    return model

denseNet = my_densenet()
denseNet.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                #metrics=['accuracy'])
                metrics=[keras.metrics.SparseCategoricalAccuracy()])
#denseNet.summary()
#keras.utils.plot_model(denseNet, 'my_denseNet.png', show_shapes=True)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
train_datagen = image.ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,  # 角度值，0-180.表示图像随机旋转的角度范围
        width_shift_range=0.2,  # 平移比例，下同
        height_shift_range=0.2,
        shear_range=0.2,  # 随机错切变换角度
        zoom_range=0.2,  # 随即缩放比例
        horizontal_flip=True,  # 随机将一半图像水平翻转
        fill_mode='nearest'  # 填充新创建像素的方法
    )
test_datagen = image.ImageDataGenerator(rescale=1 / 255)
validation_datagen = image.ImageDataGenerator(rescale=1 / 255)
train_generator = train_datagen.flow(x_train[:45000], y_train[:45000], batch_size=128)
# train_generator = train_datagen.flow(x_train, y_train, batch_size=128)
validation_generator = validation_datagen.flow(x_train[45000:], y_train[45000:], batch_size=128)
test_generator = test_datagen.flow(x_test, y_test, batch_size=128)
history = denseNet.fit_generator(train_generator, steps_per_epoch=351, epochs=100,
            validation_data=validation_generator, validation_steps=39, initial_epoch = 0)
result = denseNet.evaluate_generator(test_generator, verbose=2)
print(result)

def show_result(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.legend(['loss', 'val_loss', 'sparse_categorical_accuracy', 'val_sparse_categorical_accuracy'],
                   loc='upper left')
    plt.show()
    print(history)
show_result(history)

/shared-libs/python3.8/py/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/100
  8/351 [..............................] - ETA: 54:42 - loss: 2.2969 - sparse_categorical_accuracy: 0.1465

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import time as time
import tensorflow.keras.preprocessing.image as image
import matplotlib.pyplot as plt
import os

def dense_block(x, blocks, name, growth_rate = 32):
    for i in range(blocks):
        x = conv_block(x, growth_rate, name=name + '_block' + str(i + 1))
    return x

def transition_block(x, reduction, name):
    x = layers.BatchNormalization(axis=3, epsilon=1.001e-5,name=name + '_bn')(x)
    x = layers.Activation('relu', name=name + '_relu')(x)
    filter = x.shape[3]
    x = layers.Conv2D(int(filter*reduction), 1,use_bias=False,name=name + '_conv')(x)
    x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def conv_block(x, growth_rate, name):
    x1 = layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x)
    x1 = layers.Activation('relu')(x1)
    x1 = layers.Conv2D(2 * growth_rate, 1,use_bias=False, name=name + '_1_conv')(x1)
    x1 = layers.BatchNormalization(axis=3, epsilon=1.001e-5)(x1)
    x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
    x1 = layers.Conv2D(growth_rate, 3 ,padding='same',use_bias=False, name=name + '_2_conv')(x1)
    x = layers.Concatenate( name=name + '_concat')([x, x1])
    return x

def my_densenet():
    inputs = keras.Input(shape=(32, 32, 3), name='img')
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    blocks = [4,8,16]
    x = dense_block(x, blocks[0], name='conv1',growth_rate =32)
    x = transition_block(x, 0.5, name='pool1')
    x = dense_block(x, blocks[1], name='conv2',growth_rate =32)
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, blocks[2], name='conv3',growth_rate =32)
    x = transition_block(x, 0.5, name='pool3')
    x = layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='bn')(x)
    x = layers.Activation('relu', name='relu')(x)

    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(10, activation='softmax', name='fc1000')(x)

    model = keras.Model(inputs, x, name='densenet121')
    return model
def my_model():
    denseNet = my_densenet()

    denseNet.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                #metrics=['accuracy'])
                metrics=[keras.metrics.SparseCategoricalAccuracy()])
    denseNet.summary()
    #keras.utils.plot_model(denseNet, 'my_denseNet.png', show_shapes=True)
    return denseNet

current_max_loss = 9999

weight_file='./weights5_2/model.h5'

def train_my_model(deep_model):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    train_datagen = image.ImageDataGenerator(
        rescale=1 / 255,
        rotation_range=40,  # 角度值，0-180.表示图像随机旋转的角度范围
        width_shift_range=0.2,  # 平移比例，下同
        height_shift_range=0.2,
        shear_range=0.2,  # 随机错切变换角度
        zoom_range=0.2,  # 随即缩放比例
        horizontal_flip=True,  # 随机将一半图像水平翻转
        fill_mode='nearest'  # 填充新创建像素的方法
    )

    test_datagen = image.ImageDataGenerator(rescale=1 / 255)

    validation_datagen = image.ImageDataGenerator(rescale=1 / 255)

    train_generator = train_datagen.flow(x_train[:45000], y_train[:45000], batch_size=128)
    # train_generator = train_datagen.flow(x_train, y_train, batch_size=128)
    validation_generator = validation_datagen.flow(x_train[45000:], y_train[45000:], batch_size=128)

    test_generator = test_datagen.flow(x_test, y_test, batch_size=128)

    begin_time = time.time()

    if os.path.isfile(weight_file):
        print('load weight')
        deep_model.load_weights(weight_file)


    def save_weight(epoch, logs):
        global current_max_loss
        if(logs['val_loss'] is not None and  logs['val_loss']< current_max_loss):
            current_max_loss = logs['val_loss']
            print('save_weight', epoch, current_max_loss)
            deep_model.save_weights(weight_file)

    batch_print_callback = keras.callbacks.LambdaCallback(
        on_epoch_end=save_weight
    )
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=4, monitor='loss'),
        batch_print_callback,
        # keras.callbacks.ModelCheckpoint('./weights/model.h5', save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir='logs5_2')
    ]

    print(train_generator[0][0].shape)

    history = deep_model.fit_generator(train_generator, steps_per_epoch=351, epochs=200, callbacks=callbacks,
                                       validation_data=validation_generator, validation_steps=39, initial_epoch = 0)
    global current_max_loss
    if (history.history['val_loss'] is not None and history.history['val_loss'] < current_max_loss):
        current_max_loss = history['val_loss']
        print('save_weight', current_max_loss)
        deep_model.save_weights(weight_file)

    result = deep_model.evaluate_generator(test_generator, verbose=2)

    print(result)
    print('time', time.time() - begin_time)

    def show_result(history):
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.plot(history.history['sparse_categorical_accuracy'])
        plt.plot(history.history['val_sparse_categorical_accuracy'])
        plt.legend(['loss', 'val_loss', 'sparse_categorical_accuracy', 'val_sparse_categorical_accuracy'],
                   loc='upper left')
        plt.show()
        print(history)

    show_result(history)
def test_module(deep_model):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    test_datagen = image.ImageDataGenerator(rescale=1 / 255)


    test_generator = test_datagen.flow(x_test, y_test, batch_size=128)

    begin_time = time.time()

    if os.path.isfile(weight_file):
        print('load weight')
        deep_model.load_weights(weight_file)

    result = deep_model.evaluate_generator(test_generator, verbose=2)

    print(result)
    print('time', time.time() - begin_time)

def predict_module(deep_model):
    x_train, y_train, x_test, y_test = image_augment.get_all_train_data(False)

    import numpy as np
    if os.path.isfile(weight_file):
        print('load weight')
        deep_model.load_weights(weight_file)

    print(y_test[0:20])
    for i in range(20):
        img = x_test[i][np.newaxis, :]/255

        y_ = deep_model.predict(img)
        v  = np.argmax(y_)
        print(v, y_test[i])



#my_densenet()
#deep_model = my_model()
#train_my_model(deep_model)
#test_module(deep_model)
#predict_module(deep_model)



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=26e5d5b4-a604-4f7b-aef4-7756f8127ef0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>